<a href="https://colab.research.google.com/github/loryfelnunez/CProjects/blob/master/AgenticRag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ============================================================================
# STEP 1: Install Required Packages
# ============================================================================
!pip install -q langchain langchain-community langchain-openai
!pip install -q faiss-cpu
!pip install -q sentence-transformers
!pip install -q gradio
!pip install -q beautifulsoup4
!pip install -q openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 54.0 MB/s eta 0:00:00


In [3]:
!pip install -q requests==2.32.5

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [4]:
# ============================================================================
# STEP 2: Import Libraries
# ============================================================================
import os
from getpass import getpass
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.chains.question_answering import load_qa_chain
import gradio as gr

In [6]:
# ============================================================================
# STEP 3: Configuration
# ============================================================================
# Set your OpenAI API key
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key: ")

# Configuration parameters
WEBPAGE_URL = "https://www.apple.com/iphone/"  # Replace with your target webpage
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200
MODEL_NAME = "gpt-3.5-turbo"


Enter your OpenAI API Key: ··········


In [7]:
# ============================================================================
# STEP 4: Load Data from Webpage
# ============================================================================
print("=" * 80)
print("STEP 1: Loading webpage...")
print("=" * 80)

def load_webpage(url):
    """Load content from a webpage using WebBaseLoader"""
    loader = WebBaseLoader(url)
    documents = loader.load()
    print(f"✓ Loaded {len(documents)} document(s)")
    print(f"✓ Total characters: {len(documents[0].page_content)}")
    return documents

documents = load_webpage(WEBPAGE_URL)

STEP 1: Loading webpage...
✓ Loaded 1 document(s)
✓ Total characters: 68313


In [9]:
# ============================================================================
# STEP 5: Chunk the Documents
# ============================================================================
print("\n" + "=" * 80)
print("STEP 2: Chunking documents...")
print("=" * 80)

def chunk_documents(documents, chunk_size=1000, chunk_overlap=200):
    """
    Split documents into chunks using RecursiveCharacterTextSplitter
    This keeps semantically related text together
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )

    chunks = text_splitter.split_documents(documents)
    print(f"✓ Split into {len(chunks)} chunks")
    return chunks

chunks = chunk_documents(documents, CHUNK_SIZE, CHUNK_OVERLAP)

# Display sample chunk
print(f"\nSample chunk (first 300 characters):")
print("-" * 80)
print(chunks[0].page_content[:300] + "...")
print("-" * 80)



STEP 2: Chunking documents...
✓ Split into 99 chunks

Sample chunk (first 300 characters):
--------------------------------------------------------------------------------
iPhone - Apple
























AppleStoreMaciPadiPhoneWatch
VisionAirPodsTV & HomeEntertainmentAccessoriesSupport


0+










Get credit toward iPhone 17, iPhone Air, or iPhone 17 Pro when you trade in an eligible smartphone.Get credit toward iPhone 17, iPhone Air, or iPhone 17 Pro when y...
--------------------------------------------------------------------------------


In [10]:
# ============================================================================
# STEP 6: Generate Embeddings
# ============================================================================
print("\n" + "=" * 80)
print("STEP 3: Generating embeddings...")
print("=" * 80)

def create_embeddings():
    """
    Create embeddings using HuggingFace sentence transformers
    Using 'all-MiniLM-L6-v2' - a lightweight and efficient model
    """
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': True}
    )
    print("✓ Embeddings model loaded: all-MiniLM-L6-v2")
    return embeddings

embeddings = create_embeddings()


STEP 3: Generating embeddings...


/tmp/ipython-input-3750194709.py:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✓ Embeddings model loaded: all-MiniLM-L6-v2


In [11]:
# ============================================================================
# STEP 7: Create Vector Store
# ============================================================================
print("\n" + "=" * 80)
print("STEP 4: Creating vector store...")
print("=" * 80)

def create_vector_store(chunks, embeddings):
    """Create FAISS vector store from document chunks"""
    vectorstore = FAISS.from_documents(chunks, embeddings)
    print(f"✓ Vector store created with {len(chunks)} vectors")
    return vectorstore

vectorstore = create_vector_store(chunks, embeddings)


STEP 4: Creating vector store...
✓ Vector store created with 99 vectors


In [12]:
# ============================================================================
# STEP 8: Create Custom Prompt Template
# ============================================================================
print("\n" + "=" * 80)
print("STEP 5: Setting up prompts...")
print("=" * 80)

# Custom prompt for answering questions
qa_template = """You are a helpful AI assistant. Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.

Context: {context}

Question: {question}

Helpful Answer:"""

QA_PROMPT = PromptTemplate(
    template=qa_template,
    input_variables=["context", "question"]
)

print("✓ Custom prompts configured")



STEP 5: Setting up prompts...
✓ Custom prompts configured


In [13]:
# ============================================================================
# STEP 9: Create Conversational RAG Chain
# ============================================================================
print("\n" + "=" * 80)
print("STEP 6: Creating conversational RAG chain...")
print("=" * 80)

def create_conversational_chain(vectorstore, model_name="gpt-3.5-turbo"):
    """
    Create a conversational retrieval chain that:
    1. Maintains chat history
    2. Reformulates follow-up questions based on history
    3. Retrieves relevant context
    4. Generates answers using the LLM
    """
    # Initialize LLM
    llm = ChatOpenAI(
        model_name=model_name,
        temperature=0.7
    )

    # Create memory to store conversation history
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True,
        output_key="answer"
    )

    # Create retriever from vector store
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 4}  # Retrieve top 4 most relevant chunks
    )

    # Create conversational chain
    conversational_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory,
        return_source_documents=True,
        combine_docs_chain_kwargs={"prompt": QA_PROMPT},
        verbose=False
    )

    print("✓ Conversational RAG chain created")
    return conversational_chain

conversational_chain = create_conversational_chain(vectorstore, MODEL_NAME)



STEP 6: Creating conversational RAG chain...
✓ Conversational RAG chain created


/tmp/ipython-input-2508558160.py:23: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [15]:
# ============================================================================
# STEP 10: Test the System
# ============================================================================
print("\n" + "=" * 80)
print("STEP 7: Testing the system...")
print("=" * 80)

def test_system():
    """Test the conversational system with sample queries"""
    test_queries = [
        "What iPhones are available?",
        "What is the price of the newest one?",
        "Tell me about its camera features"
    ]

    print("\nRunning test queries:")
    print("-" * 80)

    for i, query in enumerate(test_queries, 1):
        print(f"\nQuery {i}: {query}")
        result = conversational_chain({"question": query})
        print(f"Answer: {result['answer']}")
        print("-" * 80)

# Uncomment to run tests
test_system()


STEP 7: Testing the system...

Running test queries:
--------------------------------------------------------------------------------

Query 1: What iPhones are available?
Answer: There are iPhone 13 models, iPhone 13 Pro models, and iPhone SE available.
--------------------------------------------------------------------------------

Query 2: What is the price of the newest one?
Answer: The price for the newest iPhone available includes a $30 Verizon connectivity discount. Additional terms and restrictions may apply. For the most accurate pricing information, it is recommended to speak with a Specialist at an Apple store.
--------------------------------------------------------------------------------

Query 3: Tell me about its camera features
Answer: The newest iPhone features advanced cameras that automatically capture phenomenal photos with great detail and color. The Center Stage front camera provides more flexible ways to frame selfies. The camera system offers a wide range of 

In [17]:
# ============================================================================
# STEP 11: Create Gradio Interface
# ============================================================================
print("\n" + "=" * 80)
print("STEP 8: Creating Gradio UI...")
print("=" * 80)

def create_gradio_interface():
    """Create an interactive Gradio interface for the chatbot"""

    # Global variable to store the chain (in production, use proper state management)
    global conversational_chain

    def chat(message):
        """
        Process user message and return response

        Args:
            message: User's current message

        Returns:
            Updated response for display
        """
        try:
            # Query the conversational chain
            result = conversational_chain({"question": message})

            # Extract the answer
            answer = result['answer']

            # Optionally, you can add source information
            # sources = result.get('source_documents', [])
            # if sources:
            #     answer += "\n\nSources: " + ", ".join([doc.metadata.get('source', 'N/A') for doc in sources[:2]])

            return answer

        except Exception as e:
            return f"Sorry, I encountered an error: {str(e)}"

    def reset_conversation():
        """Reset the conversation history"""
        global conversational_chain
        conversational_chain.memory.clear()
        return None

    # Create Gradio interface
    with gr.Blocks(title="RAG Chatbot", theme=gr.themes.Soft()) as demo:
        gr.Markdown(
            """
            # 🤖 RAG-Powered Conversational Chatbot

            Ask questions about the loaded webpage content. The system remembers context from previous questions!

            **Example conversation:**
            - "What products are available?"
            - "What is the price of the newest one?"
            - "Tell me more about its features"
            """
        )

        chatbot = gr.ChatInterface(
            fn=chat,
            examples=[
                "What information is available on this page?",
                "Can you summarize the main content?",
                "What are the key features mentioned?"
            ],
            title=""
        )

        with gr.Row():
            gr.Markdown(
                """
                ### Tips:
                - Ask follow-up questions naturally (e.g., "What about the price?")
                - The system remembers previous questions in the conversation
                - Click "Clear Conversation" to start fresh
                """
            )

    print("✓ Gradio interface created")
    return demo

# Create and launch the interface
demo = create_gradio_interface()

print("\n" + "=" * 80)
print("Launching Gradio interface...")
print("=" * 80)

# Launch the app
demo.launch(share=True, debug=False)


STEP 8: Creating Gradio UI...


/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


✓ Gradio interface created

Launching Gradio interface...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0c400070a713078d69.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
